<a href="https://colab.research.google.com/github/phitonthel/YOG1-C_Final_Project/blob/master/Resnet_WITH_MINIMAL_Image_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"

In [ ]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

# List available datasets.
!kaggle datasets list



In [ ]:
# Download the Kaggle Dataset
!kaggle datasets download -d luisblanche/covidct

In [ ]:
import zipfile
base_dir = '/content'
data_dir = os.path.join(base_dir, 'data')
local_zip = os.path.join(base_dir, 'covidct.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
os.mkdir(data_dir)
zip_ref.extractall(data_dir)
zip_ref.close()

In [ ]:
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.mkdir(train_dir)
os.makedirs(os.path.join(test_dir, 'CT_COVID'))
os.mkdir(os.path.join(test_dir, 'CT_NonCOVID'))

In [ ]:
import random
import math
import shutil

train_portion = .90

def splitter(name):
  data_path = os.path.join(data_dir, name)
  temp = os.listdir(data_path)
  random.shuffle(temp)
  split = math.floor(len(temp) * train_portion)

  test_tmp = temp[split:]

  for i in test_tmp:
    shutil.move(os.path.join(data_path, i), os.path.join(test_dir, name, i))
  
  shutil.move(data_path, train_dir)

In [ ]:
splitter('CT_COVID')
splitter('CT_NonCOVID')

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=20,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    # shear_range = 0.1,
    # fill_mode = 'nearest'
    )
     
test_datagen = ImageDataGenerator(
    rescale=1./255
    )

train_generator = train_datagen.flow_from_directory(
	train_dir,
	target_size=(150,150),
	class_mode='binary',
  batch_size= 61
)

test_generator = test_datagen.flow_from_directory(
	test_dir,
	target_size=(150,150),
	class_mode='binary',
  batch_size=25
)

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape=(150, 150, 3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
regularizer = tf.keras.regularizers.L1L2(
    l1=0.5, l2=0.5
)
for layer in base_model.layers:
  layer.trainable = False
  for attr in ['kernel_regularizer']:
    if hasattr(layer, attr):
      setattr(layer, attr, regularizer)

In [ ]:
base_model.summary()

In [ ]:
last_layer = base_model.get_layer('conv5_block3_out')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
class AccCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > .9:
      self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(.2)(x)                  
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)           

model = tf.keras.Model(base_model.input, x) 

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(train_generator, epochs=40, steps_per_epoch=3, validation_data = test_generator, verbose = 1, validation_steps=3,
                    # callbacks=[AccCallback()]
                    )

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Testing Accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, loss, 'g', label='Loss Training')
plt.plot(epochs, val_loss, 'c', label='Loss Testing')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model

saved_model_dir = 'saved_model/my_model'
model.save(saved_model_dir) 

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)


In [ ]:
model.export(export_dir='.')